In [1]:
# 구글 드라이브 사용 권한 설정
import os 
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive/Colab Notebooks/Capstone'

/gdrive/My Drive/Colab Notebooks/Capstone


# Test

In [3]:
#깃허브 클론
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git
%cd '/gdrive/My Drive/Colab Notebooks/Capstone/deep-text-recognition-benchmark'

fatal: destination path 'deep-text-recognition-benchmark' already exists and is not an empty directory.
/gdrive/My Drive/Colab Notebooks/Capstone/deep-text-recognition-benchmark


In [4]:


models = {
    'None-ResNet-None-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
    'None-VGG-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1GGC2IRYEMQviZhqQpbtpeTgHO_IXWetG',
    'None-VGG-None-CTC.pth': 'https://drive.google.com/open?id=1FS3aZevvLiGF1PFBm5SkwvVcgI6hJWL9',
    'TPS-ResNet-BiLSTM-Attn-case-sensitive.pth': 'https://drive.google.com/open?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY',
    'TPS-ResNet-BiLSTM-Attn.pth': 'https://drive.google.com/open?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9',
    'TPS-ResNet-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
}

for k, v in models.items():
  doc_id = v[v.find('=')+1:]
  !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
  !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2288    0  2288    0     0   7477      0 --:--:-- --:--:-- --:--:--  7501
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   559  100   559    0     0   1565      0 --:--:-- --:--:-- --:--:--  1565
100  147k    0  147k    0     0   292k      0 --:--:-- --:--:-- --:--:--  292k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  % Total    % Received % Xferd  Average Speed   Tim

In [4]:
output = !CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

In [ ]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

data = pd.DataFrame()
for ind, row in enumerate(output[output.index('image_path               \tpredicted_labels         \tconfidence score')+2:]):
  row = row.split('\t')
  filename = row[0].strip()
  label = row[1].strip()
  conf = row[2].strip()
  img = Image.open(filename)
  img_buffer = io.BytesIO()
  img.save(img_buffer, format="PNG")
  imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8") 

  data.loc[ind, 'img'] = ''.format(imgStr)
  data.loc[ind, 'id'] = filename
  data.loc[ind, 'label'] = label
  data.loc[ind, 'conf'] = conf

html_all = data.to_html(escape=False)
display(HTML(html_all))

# Train

In [7]:

import os
import sys
import time
import random
import string
import argparse

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
# from dataset.mnist import load_mnist
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np
from PIL import Image


from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
from model import Model
from mytest import validation   # python 라이브러리 중에 unittest라는 라이브러리가 존재한다. 따라서 test.py의 이름 mytest로 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, one_hot_label=False, )

In [10]:
t_train[0:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [25]:
# ### mnist image 
# for i,img in enumerate(x_train,1):
#   img_2 = Image.fromarray(img.reshape(28,28)) # NumPy array to PIL image
#   txt = format(i,'06')
#   img_2.save('./mnistimage/'+txt+'.jpg','JPEG') # save PIL image

In [15]:
# ## gt.txt 만들기 
# f = open("./gt.txt", "w")
# for i,label in enumerate(t_train,1):
#   txt = format(i,'06')
#   f.write('mnistimage/'+txt+'.jpg\t'+str(label)+'\n')
# f.close()

In [ ]:
# #학습데이터 lmdb 포맷으로 
# !pip install lmdb

In [17]:
# !pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# !python3 create_lmdb_dataset.py \
#     --inputPath ./ \
#     --gtFile ./gt.txt \
#     --outputPath ./result/train/

Written 1000 / 60000
Written 2000 / 60000
Written 3000 / 60000
Written 4000 / 60000
Written 5000 / 60000
Written 6000 / 60000
Written 7000 / 60000
Written 8000 / 60000
Written 9000 / 60000
Written 10000 / 60000
Written 11000 / 60000
Written 12000 / 60000
Written 13000 / 60000
Written 14000 / 60000
Written 15000 / 60000
Written 16000 / 60000
Written 17000 / 60000
Written 18000 / 60000
Written 19000 / 60000
Written 20000 / 60000
Written 21000 / 60000
Written 22000 / 60000
Written 23000 / 60000
Written 24000 / 60000
Written 25000 / 60000
Written 26000 / 60000
Written 27000 / 60000
Written 28000 / 60000
Written 29000 / 60000
Written 30000 / 60000
Written 31000 / 60000
Written 32000 / 60000
Written 33000 / 60000
Written 34000 / 60000
Written 35000 / 60000
Written 36000 / 60000
Written 37000 / 60000
Written 38000 / 60000
Written 39000 / 60000
Written 40000 / 60000
Written 41000 / 60000
Written 42000 / 60000
Written 43000 / 60000
Written 44000 / 60000
Written 45000 / 60000
Written 46000 / 600

In [11]:
#####검증 데이터 만들기 
for i,img in enumerate(x_test,1):
  img_2 = Image.fromarray(img.reshape(28,28)) # NumPy array to PIL image
  txt = format(i,'06')
  img_2.save('./valid_img/'+txt+'.jpg','JPEG') # save PIL image

In [12]:
# ## gt.txt 만들기 
f = open("./valid_gt.txt", "w")
for i,label in enumerate(t_test,1):
  txt = format(i,'06')
  f.write('valid_img/'+txt+'.jpg\t'+str(label)+'\n')
f.close()

In [13]:
# #학습데이터 lmdb 포맷으로 
!pip install lmdb
!pip install fire

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=198b91409197f41384148906c15ef125bac4ded83d3d25100c77bd8a62668253
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire


In [14]:
!python3 create_lmdb_dataset.py \
    --inputPath ./ \
    --gtFile ./valid_gt.txt \
    --outputPath ./result/validation/

Written 1000 / 10000
Written 2000 / 10000
Written 3000 / 10000
Written 4000 / 10000
Written 5000 / 10000
Written 6000 / 10000
Written 7000 / 10000
Written 8000 / 10000
Written 9000 / 10000
Written 10000 / 10000
Created dataset with 10000 samples


In [ ]:
!python3 train.py --train_data ./result/train \
    --valid_data ./result/validation \
    --Transformation TPS \
    --FeatureExtraction ResNet \
    --SequenceModeling BiLSTM \
    --Prediction Attn \
    --saved_model models/TPS-ResNet-BiLSTM-Attn.pth \
    --FT

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: ./result/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    ./result/train	 dataset: /
sub-directory:	/.	 num samples: 60000
num total samples of /: 60000 x 1.0 (total_data_usage_ratio) = 60000
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slow